2021-06-10

个人作业：
1. 向量计算的瓶颈是什么？
2. 可以使用什么工具来查看CPU执行计算的效率？
3. 为什么Numpy比纯Python更擅长数值计算？
4. 什么是缓存未命中和页面错误？
5. 如何跟踪代码中的内存分配？
（以上周四提交）
练习2：Numpy100题（周五提交）
练习3：Scipy 与统计分析基础（周六提交）
练习4：Pandas训练50题（周日提交）

小组作业：（本期小组作业可以延后至下周一晚10点提交）
安装linux环境，使用Perf性能工具复现"2 矩阵和向量计算"讲义内容

### 向量计算的瓶颈是什么？
+ Python本身并不支持向量化。因为Python列表存储指向实际数据的指针；Python底层字节码没有针对向量化进行优化，因此，for 循环无法预测何时使用向量化是有益的。每次我们要从矩阵中提取一个元素时，都必须进行多次查找，这会导致性能下降。同时当数据碎片化时，必须逐个移动每个片段，而不是移动整个块。这意味着调用了更多的内存传输开销，并且在传输数据时强制CPU等待。
+ 现代计算机使用的分层内存体系结构，内存和CPU之间存在的有限带宽。

### 可以使用什么工具来查看CPU执行计算的效率？
Perf 提供给我们的各种性能指标以及它们与代码的关系，查看CPU运行代码的效率。

### 为什么Numpy比纯Python更擅长数值计算？
+ Numpy将数据存储在连续的内存块中，并支持对其数据的向量化操作。因此,对 Numpy 数组所做的任何运算都是成片进行的，而不必显式地循环每个元素。这样做矩阵运算不仅容易得多，而且速度更快。
+ 尽管 Numpy 版本似乎每个周期运行的指令更少，但每条指令都要做更多的工作。也就是说，一条向量化指令可以将一个数组中的四个（或更多）数字相乘，而不需要四条独立的乘法指令。总的来说，这会减少解决同一问题所需的全部指令。
+ 其他几个因素导致 Numpy 版本需要较低的指令绝对数来解决扩散问题。其中之一与运行纯 Python 版本时可用的完整 Python API有关，但不一定是对于 Numpy 版本，例如，纯Python网格可以在纯Python中附加，但不能在 Numpy 中附加。尽管我们没有显式地使用这个（或其他）功能，但是在提供一个可以使用它的系统时会有开销。由于 Numpy 可以假设存储的数据总是数字，因此可以针对数字上的操作对数组的所有内容进行优化。当我们谈论 Cython（参见“Cython”）时，我们将继续删除有利于性能的必要功能，在Cython中甚至可以删除列表边界检查来加速列表查找。
+ 通常情况下，指令的数量不一定与性能相关。指令较少的程序可能无法有效地发出指令，或者它们可能是慢指令。然而，我们看到，除了减少指令数量之外，Numpy版本还降低了一个很大的低效率：缓存未命中（缓存未命中率为20.8%，而不是53.3%）。如“内存碎片”中所述，缓存未命中会降低计算速度，因为CPU必须等待从较慢的内存中检索数据，而不是让数据立即在其缓存中可用。事实上，内存碎片是影响性能的一个主要因素，如果我们在 Numpy 中禁用矢量化，但保持其他所有内容不变，与纯Python版本相比，我们仍然可以看到相当大的速度提高

### 什么是缓存未命中和页面错误？
+ 每当处理器想要从主存储器获取数据时，首先它将查看缓存缓冲区以查看缓冲区中是否存在相应的地址。如果它在那里，它将使用缓存执行操作，无需从主内存中获取。这被称为“缓存命中”，而如果缓存中不存在该地址，则称其为“缓存未命中”。
+ 页面错误：当软件试图读取或写入标记为“不存在”的虚拟内存位置时发生的中断。页面错误记录了一个进程必须从硬盘上恢复的次数。

### 如何跟踪代码中的内存分配？
1. 询问操作系统
    + 可以使用top来提供在一段时间内使用的资源的概述
    + 如果想要现场检查资源使用情况，结合一些创造性的shell脚本，可以编写一个监视脚本，使用ps跟踪任务的内存使用情况。
2. 利用tracemalloc模块
    + tracemalloc可以跟踪Python解释器分配的每个单独的内存块。tracemalloc能够提供关于运行Python进程中内存分配的非常细粒度的信息:
        ~~~
        import tracemalloc
        tracemalloc.start()
        my_complex_analysis_method()
        current, peak = tracemalloc.get_traced_memory()
        print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")
        tracemalloc.stop()
        
3. 利用resource模块
    + resource模块为程序分配的资源提供基本控制，包括内存使用
    ~~~ 
    import resource
    usage = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss

